In [ ]:
import robotic as ry
import numpy as np
import time
print('version:', ry.__version__, ry.compiled()) # mainpulation.o;rg checkpointleri rrt den alın, hepsini almanıza gerek yok.

version: 0.2.2 compile time: Nov 26 2024 16:47:22


In [2]:
C = ry.Config()
C.addFile("./custom_maze.g")
#C.view()

In [3]:
#C.getFrameNames()

In [ ]:
 # rrt dene slicesperphase = 1
# yerle collision çözülecek
# github hesabı açılacak.

komo = ry.KOMO(C, phases=7, slicesPerPhase = 20, kOrder = 2, enableCollisions = True) 

komo.addControlObjective([], 2, 1e0) # penalize acceleration
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.sos, [1e2]) # avoid collisions  # no sos here.


###### Robot 1 stages ######
# go near the cargo
komo.addObjective([1.0, 2.8], ry.FS.distance, ["ego","cargo"],ry.OT.eq, [1e2])

# grasping phase - make it stable.
komo.addModeSwitch([1.3, 3], ry.SY.stable, ["ego", "cargo"], True) 

# move it to the passage_point
komo.addObjective([2, 3], ry.FS.distance, ["cargo","passage_line"],ry.OT.eq, [1e2]) 
"""
##slow - down - up
komo.addObjective([3.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([2.9,3.1], ry.FS.position, ["l_gripper"], ry.OT.eq, [], [0.,0.,.1], 2)
"""
#place
komo.addModeSwitch([3., 4.], ry.SY.stable, [ "passage","cargo"], True)
komo.addObjective([3., 4.], ry.FS.positionDiff, ["cargo", "passage"], ry.OT.eq,  [1e2, 1e2, 0])
#komo.addObjective([3., 4.], ry.FS.vectorZ, ["l_gripper"], ry.OT.eq, [1e2], [0., 0., 1.])


###### Robot 2 stages ######
# grasping 
komo.addObjective([4.5,6], ry.FS.distance, ["ego2","cargo"],ry.OT.eq, [1e2])

komo.addModeSwitch([4.7,6], ry.SY.stable, ["ego2", "cargo"], True) # make it stable.

# move it to the goal area
komo.addObjective([5,-1], ry.FS.positionDiff, ["cargo","goal_area"],ry.OT.eq,[1e2, 1e2, 0])

komo.addModeSwitch([6.1, -1], ry.SY.stable, ["goal_area", "cargo"], True) 


"""
#slow - down - up
komo.addObjective([7.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([6.9,7.1], ry.FS.position, ["l_gripper"], ry.OT.eq, [], [0.,0.,.1], 2)

#place
komo.addModeSwitch([7., 8.], ry.SY.stable, ["floorwalls", "box"])
komo.addObjective([7., 8.], ry.FS.positionDiff, ["box", "table"], ry.OT.eq, [1e2], [0,0,.08])
komo.addObjective([7., 8.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])
"""


solver = ry.NLP_Solver(komo.nlp(),verbose = 4)
for i in range(10):
    ret = solver.solve()
    if ret.feasible:
        print("feasible")
        break
print(ret)
print(ret.feasible)
komo.view_play(True, f"Solution, {ret.feasible}, {ret.eq}")
q = komo.getPath()
print(q.shape)


In [ ]:

komo = ry.KOMO(C, phases=7.3, slicesPerPhase = 20, kOrder = 2, enableCollisions = True) 

komo.addControlObjective([], 2, 1e0) # penalize acceleration
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2]) # avoid collisions  # no sos here.


###### Robot 1 stages ######
# go near the cargo
komo.addObjective([1.0, 2.8], ry.FS.distance, ["l_gripper","cargo"],ry.OT.eq, [1e2])

# grasping phase - make it stable.
komo.addModeSwitch([1.3, 3], ry.SY.stable, ["l_gripper", "cargo"], True) 

# move it to the passage_point
komo.addObjective([2, 3], ry.FS.distance, ["cargo","passage"],ry.OT.eq, [1e2]) 
"""
##slow - down - up
komo.addObjective([3.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([2.9,3.1], ry.FS.position, ["l_gripper"], ry.OT.eq, [], [0.,0.,.1], 2)
"""
#place
komo.addModeSwitch([3., 4.], ry.SY.stable, [ "passage","cargo"], True)
komo.addObjective([3., 4.], ry.FS.distance, ["cargo", "passage"], ry.OT.eq, [1e2])
#komo.addObjective([3., 4.], ry.FS.vectorZ, ["l_gripper"], ry.OT.eq, [1e2], [0., 0., 1.])


###### Robot 2 stages ######
# grasping 
komo.addObjective([4.5,7], ry.FS.distance, ["r_gripper","cargo"],ry.OT.eq, [1e2])

komo.addModeSwitch([4.7,6.8], ry.SY.stable, ["r_gripper", "cargo"], True) # make it stable.

# move it to the goal area
komo.addObjective([5,7], ry.FS.distance, ["cargo","goal_area"],ry.OT.eq, [1e2])

"""
#slow - down - up
komo.addObjective([7.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([6.9,7.1], ry.FS.position, ["l_gripper"], ry.OT.eq, [], [0.,0.,.1], 2)

#place
komo.addModeSwitch([7., 8.], ry.SY.stable, ["floorwalls", "box"])
komo.addObjective([7., 8.], ry.FS.positionDiff, ["box", "table"], ry.OT.eq, [1e2], [0,0,.08])
komo.addObjective([7., 8.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])
"""

solver = ry.NLP_Solver(komo.nlp(),verbose = 4)
ret = solver.solve()
print(ret)
print(ret.feasible)
komo.view_play(True, f"Solution, {ret.feasible}, {ret.eq}")
q = komo.getPath()
print(q.shape)


In [ ]:
# First successful attempt
komo = ry.KOMO(C, phases=8, slicesPerPhase = 20, kOrder = 2, enableCollisions = True) 

komo.addControlObjective([], 2, 1e0) # penalize acceleration
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2]) # avoid collisions  # no sos here.


###### Robot 2 stages ######
# go near the cargo
komo.addObjective([1.0, 3], ry.FS.distance, ["l_gripper","cargo_handle"],ry.OT.eq, [1e2])

# grasping phase - make it stable.
komo.addModeSwitch([1.3, 3], ry.SY.stable, ["l_gripper", "cargo_handle"], True) 

# move it to the passage_point
komo.addObjective([2, 3], ry.FS.distance, ["cargo","passage"],ry.OT.eq, [1e2]) 



###### Robot 2 stages ######
# grasping 
komo.addObjective([3.2,7], ry.FS.distance, ["r_gripper","cargo_handle"],ry.OT.eq, [1e2])

komo.addModeSwitch([3.5,6.8], ry.SY.stable, ["r_gripper", "cargo_handle"], True) # make it stable.

# move it to the goal area
komo.addObjective([4,7], ry.FS.distance, ["cargo","goal_area"],ry.OT.eq, [1e2])


# OPTIONAL - leave the cargo


solver = ry.NLP_Solver(komo.nlp(),verbose = 4)
ret = solver.solve()
print(ret)
print(ret.feasible)
komo.view_play(True, f"Solution, {ret.feasible}, {ret.eq}")
q = komo.getPath()
print(q.shape)


{ time: 3.82025, evals: 72, done: 1, feasible: 0, eq: 9162.19, ineq: 0, sos: 4587.46, f: 0 }
False
(200, 20)


In [ ]:
# Report
komo.report()
print("ret.feasible:",ret.feasible)